## Project - Dögg's working file

In [43]:
import pandas as pd
import pytz

#### Data pre-processing

In [23]:
data = pd.read_csv('Ditte/EVChargingStationUsage.csv')
data.head()

/var/folders/y8/7qgmhhm95nd9c_b70tsnww880000gq/T/ipykernel_1350/186341111.py:1: DtypeWarning: Columns (29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Ditte/EVChargingStationUsage.csv')


,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),...,Longitude,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,...,-122.160309,USD,0.0,Plug Out at Vehicle,3,95124.0,3284.0,NaN,NaN,NaN
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,...,-122.160309,USD,0.0,Customer,4,94301.0,4169.0,NaN,NaN,NaN
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,...,-122.160309,USD,0.0,Plug Out at Vehicle,5,94301.0,4169.0,NaN,NaN,NaN
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,...,-122.160309,USD,0.0,Customer,6,94302.0,2545.0,NaN,NaN,NaN
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,...,-122.160309,USD,0.0,Plug Out at Vehicle,7,94043.0,3765.0,NaN,NaN,NaN


In [24]:
# Convert datetime columns to datetime objects and 
data['Start Date'] = pd.to_datetime(data['Start Date'])
data['End Date'] = pd.to_datetime(data['End Date'], errors='coerce') # some values are incorrect, they will be converted to NaT and removed later
data['Total Duration (hh:mm:ss)'] = pd.to_timedelta(data['Total Duration (hh:mm:ss)'])
data['Charging Time (hh:mm:ss)'] = pd.to_timedelta(data['Charging Time (hh:mm:ss)'])

## Cleanup

In [25]:
# View missing values
print(data.isna().sum())

Station Name                           0
MAC Address                            0
Org Name                               0
Start Date                             0
Start Time Zone                        0
End Date                              40
End Time Zone                          0
Transaction Date (Pacific Time)      209
Total Duration (hh:mm:ss)              0
Charging Time (hh:mm:ss)               0
Energy (kWh)                           0
GHG Savings (kg)                       0
Gasoline Savings (gallons)             0
Port Type                              9
Port Number                            0
Plug Type                              0
EVSE ID                            78948
Address 1                              0
City                                   0
State/Province                         0
Postal Code                            0
Country                                0
Latitude                               0
Longitude                              0
Currency        

In [26]:
# Remove columns where more than 8000 values are missing
threshold = 8000
cols_to_drop = data.columns[data.isna().sum() > threshold]
data = data.drop(columns=cols_to_drop)

# View
print(data.isna().sum())

Station Name                          0
MAC Address                           0
Org Name                              0
Start Date                            0
Start Time Zone                       0
End Date                             40
End Time Zone                         0
Transaction Date (Pacific Time)     209
Total Duration (hh:mm:ss)             0
Charging Time (hh:mm:ss)              0
Energy (kWh)                          0
GHG Savings (kg)                      0
Gasoline Savings (gallons)            0
Port Type                             9
Port Number                           0
Plug Type                             0
Address 1                             0
City                                  0
State/Province                        0
Postal Code                           0
Country                               0
Latitude                              0
Longitude                             0
Currency                           1788
Fee                                   0


In [27]:
# Remove all rows with missing values
data = data.dropna()

# View
print(data.isna().sum())

Station Name                       0
MAC Address                        0
Org Name                           0
Start Date                         0
Start Time Zone                    0
End Date                           0
End Time Zone                      0
Transaction Date (Pacific Time)    0
Total Duration (hh:mm:ss)          0
Charging Time (hh:mm:ss)           0
Energy (kWh)                       0
GHG Savings (kg)                   0
Gasoline Savings (gallons)         0
Port Type                          0
Port Number                        0
Plug Type                          0
Address 1                          0
City                               0
State/Province                     0
Postal Code                        0
Country                            0
Latitude                           0
Longitude                          0
Currency                           0
Fee                                0
Ended By                           0
Plug In Event Id                   0
U

## Data preparation

In [28]:
# Convert timezone to UTC
# I have to convert the datetime to the same timezone, as there are too many different ones.
# Unique start and end time timezone combinations
#	PDT	PDT
#	PST	PST
#	PST	PDT
#	PDT	PST
#	UTC	PDT
#	UTC	PST
#	PDT	UTC
# Therefore, it would probably be best to convert to UTC first to calculate the hourly energy use and hourly charger occupation, 
# then switch back to US/Pacific for the day vs night and weekday vs weekend clustering analysis

# Map timezones: (pytz handles daytime savings)
tz_mapping = {
    'PDT': 'US/Pacific',
    'PST': 'US/Pacific',
    'UTC': 'UTC'
}

# Function for localizing timezone (adding it to datetime column)
def localize_to_timezone(row, datetime_col, tz_col):
    tz_name = row[tz_col]  # Get timezone string
    timezone = pytz.timezone(tz_mapping.get(tz_name))  # Map to pytz timezone string
    return row[datetime_col].tz_localize(timezone, ambiguous='NaT', nonexistent='NaT') # Localize datetime

# Apply the localization function to 'Start Date' and 'End Date'
data['Start Date Localized'] = data.apply(lambda row: localize_to_timezone(row, 'Start Date', 'Start Time Zone'), axis=1)
data['End Date Localized'] = data.apply(lambda row: localize_to_timezone(row, 'End Date', 'End Time Zone'), axis=1)

# Drop rows with non-existent 'spring forward' datetimes and those with ambiguous 'fall back' datetimes
data = data.dropna()

# Convert to UTC
data['Start Date UTC'] = pd.to_datetime(data['Start Date Localized'], utc=True)
data['End Date UTC'] = pd.to_datetime(data['End Date Localized'], utc=True)

## Calculate hourly energy use and hourly charger occupation / resample

Next up is calculating the hourly energy use and the hourly charger occupation.

So, I now have my time in UTC as events. I need to convert these events to multiple rows, as I need one row per hour

These are the relevant columns from row 1, let's use this as an example:
* Total Duration (hh:mm:ss)                     0 days 03:03:32
* Charging Time (hh:mm:ss)                      0 days 01:54:03
* Energy (kWh)                                         6.249457
* Start Date UTC                      2011-07-30 03:17:00+00:00
* End Date UTC                        2011-07-30 06:20:00+00:00

The Charging Time is 01:54:03, and the Total Duration is 0 days 03:03:32. And the Start Date UTC is 2011-07-30 03:17:00+00:00, and End Date UTC is 2011-07-30 06:20:00+00:00, and the Energy (kWh) is 6.249457.

### Plan
I want the hourly energy use to be calculated like this:

1. Convert 'Charging Time (hh:mm:ss)' to minutes: 0 days 01:54:03 = 114.05 min as 'Total Charging Time (min)' and 
'Total Duration (hh:mm:ss)' to minutes: 0 days 03:03:32 = 183.53

2. Find charging per minute 'kWh per min': 'Energy (kWh)' / 'Total Charging Time (min)' = 6.249457 / 114.05 = 0.05479576502

3. This event has to be converted to 4 rows: where the 'Date UTC' is 2011-07-30 03:00:00+00:00, 2011-07-30 04:00:00+00:00, 
2011-07-30 05:00:00+00:00, and 2011-07-30 06:00:00+00:00

4. Figure out how many minutes were charging/occupied in each:
* 'Date UTC' : 'Charging Time (min)' : 'Duration (min)'
* 2011-07-30 03:00:00+00:00 : 60-17 = 43 : 60 - 17 = 43
* 2011-07-30 04:00:00+00:00 : 60-0 (if sum of charging min does not exceed 'Total Charging Time (min)') = 60 : 60 - 0 (if sum of duration min does not exceed 'Total Duration (min)')= 60
* 2011-07-30 05:00:00+00:00 : 60-0 (if sum of charging min does not exceed 'Total Charging Time (min)') = 60, but exceeds limit of 114, so Charging Time (min) = 114.05 - 17 - 60 = 37.05 : 60 - 0 (if sum of duration min does not exceed 'Total Duration (min)')= 60
* 2011-07-30 06:00:00+00:00 : 20 (if sum of charging min does not exceed 'Total Charging Time (min)') = 60, but this exceeds limit of 114, so Charging Time (min) = 114.05 - 17 - 60 - 37.05 = 0 : 20

5. This gives the following columns and rows, where we add hourly energy consumption and hourly charger occupation:
* 'Date UTC' : 'Charging Time (min)' : 'Duration (min)' : 'Hourly Energy Consumption (kWh)' : 'Hourly Charger Occupation (h)'
* 2011-07-30 03:00:00+00:00 : 43 : 43 : Charging Time (min) * kWh per min = 43 * 0.05479576502 = 2.3562178959 : 43/60 = 0.717 
* 2011-07-30 04:00:00+00:00 : 60 : 60 : Charging Time (min) * kWh per min = 60 * 0.05479576502 = 3.2877459012 : 60/60 = 1 
* 2011-07-30 05:00:00+00:00 : 37.05 : 60 : Charging Time (min) * kWh per min = 37.05 * 0.05479576502 = 2.03018309399 : 60/60 = 1
* 2011-07-30 06:00:00+00:00 : 0 : 20 : Charging Time (min) * kWh per min = 0 * 0.05479576502 = 0 : 20/60 = 0.333


In [29]:
# Step 1: Converting Charging Time and Total Duration to minutes:
data['Total Charging Time (min)'] = pd.to_timedelta(data['Charging Time (hh:mm:ss)']).dt.total_seconds() / 60
data['Total Duration (min)'] = pd.to_timedelta(data['Total Duration (hh:mm:ss)']).dt.total_seconds() / 60

In [30]:
# Step 2: Find charging per minute 'kWh per min'
data['kWh per min'] = data['Energy (kWh)'] / data['Total Charging Time (min)']

In [31]:
# Extra step: Add id column to original dataframe, so I don't have to copy all values in all rows:
data['unique_id'] = range(len(data))

In [32]:
# Step 3, 4, 5: 

# Function for counting the _ minutes. Used for charging and duration
def count_minutes(start_date, current_hour, total__time, minutes_):
    if (start_date.replace(minute=0, second=0) == current_hour):
        if ((60 - start_date.minute) > total__time):
            _min_in_hour = total__time
        else:
            _min_in_hour = 60 - start_date.minute
    elif ((minutes_ + 60) <= total__time):
        _min_in_hour = 60
    elif ((minutes_ + 60) > total__time):
        _min_in_hour = total__time - minutes_
    else:
        print("You missed something :P")
    return _min_in_hour

# Funtion for splitting events into multiple rows
def split_event_into_hours(row):
    start_date = row['Start Date UTC']
    end_date = row['End Date UTC']
    total_charging_time = row['Total Charging Time (min)']
    total_duration = row['Total Duration (min)']
    kWh_per_min = row['kWh per min']
    
    # Create a list of hours
    hourly_rows = []
    
    # Calculate minute totals
    minutes_charged = 0
    minutes_occupied = 0
    
    # Initialize current and iterate over all the hours
    current_hour = start_date.replace(minute=0, second=0)
    while current_hour < end_date:
        
        # Calculate minutes_in_hour
        charging_min_in_hour = count_minutes(start_date, current_hour, total_charging_time, minutes_charged)
        occupied_min_in_hour = count_minutes(start_date, current_hour, total_duration, minutes_occupied)
        
        # Calculate hourly energy consumption and charger occupation
        hourly_energy = charging_min_in_hour * kWh_per_min
        hourly_occupation = occupied_min_in_hour / 60  # Convert to hours
        
        # Append row
        hourly_rows.append({
            'Date UTC': current_hour,
            'Charging Time (min)': charging_min_in_hour,
            'Duration (min)': occupied_min_in_hour,
            'Hourly Energy Consumption (kWh)': hourly_energy,
            'Hourly Charger Occupation (h)': hourly_occupation,
            'Original Row ID': row['unique_id']
        })

        # Update counters
        minutes_charged += charging_min_in_hour
        minutes_occupied += occupied_min_in_hour

        # Move to the next hour
        current_hour += pd.Timedelta(hours=1)

    return pd.DataFrame(hourly_rows)

# Apply the function and create a new dataframe
new_data = pd.concat(data.apply(split_event_into_hours, axis=1).tolist(), ignore_index=True)

# Display the result

In [33]:
# Create new column with US/Pacific time
new_data['Date Pacific'] = new_data['Date UTC'].dt.tz_convert('US/Pacific')

In [34]:
# Create dummy for night and weekend

# Night: check if time is between 18 and 6
new_data['Is Night'] = ((new_data['Date Pacific'].dt.hour >= 18) | (new_data['Date Pacific'].dt.hour < 6)).astype(int)

# Weekend: check if day is 5 or 6, aka weekend
new_data['Is Weekend'] = new_data['Date Pacific'].dt.dayofweek.isin([5, 6]).astype(int)

In [39]:
# Merge the dataframes.
merged_data = pd.merge(new_data, data, left_on='Original Row ID', right_on='unique_id', how='inner')

# Remove the unique_id column, as name is misleading, and the same information is in the Original Row ID column.
merged_data = merged_data.drop(columns=['unique_id'])


In [53]:
# Cluster day vs night

### cluster weekday vs weekend